In [13]:
import requests
from collections import defaultdict
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

import re

import time

import logging

In [65]:
logger = logging.getLogger('crawler')
logger.setLevel(logging.DEBUG)

In [27]:
search_keyword = '"기본소득"'
outlets_of_interest = ['경향신문', '한겨레', '조선일보', '동아일보']

# 네이버

In [68]:
all_articles = []
data = defaultdict(list)
parse_time_per_page = []

for page_index in range(100):
    article_index = page_index * 10 + 1
    
    url = f"https://search.naver.com/search.naver?where=news&query={search_keyword}&sort=1&start={article_index}"
    
    start = time.time()
    raw = requests.get(url, headers={'User-Agent':'Mozilla/5.0'})
    
    if raw.status_code == 404:
        logger.warning("404 at {url}")
        continue
        
    html = BeautifulSoup(raw.text, "html.parser")

    articles = html.select("ul.type01 > li")
    
    for i, article in enumerate(articles):
        title = article.select_one("a._sp_each_title").text
        outlet = article.select_one("span._sp_each_source").text
        date = re.sub(outlet, '', re.sub('보내기', '', article.select_one("dd.txt_inline").text)).strip()
        
        if outlet.endswith(' 선정'):
            pick = True
        else:
            pick = False
        
        outlet = outlet.strip(' 선정')
        
        data['title'].append(title)
        data['outlet'].append(outlet)
        data['date'].append(date)
        data['pick'].append(pick)
        
    end = time.time()
    
    parse_time_per_page.append(end - start)

In [ ]:
    articles = html.select("ul.type01 > li")
    select_ids = []
    
    for i, article in enumerate(articles):
        outlet = article.select_one("span._sp_each_source").text
        outlet = outlet.strip(" 선정")
        if outlet in outlets_of_interest:
            print(outlet, end='\r')
            select_ids.append(i)
            
    all_articles.extend([articles[j] for j in select_ids])
    

In [53]:
np.max(parse_time_per_page)

5.9303319454193115

In [29]:
all_articles[0]

<li id="sp_nws12"> <dl> <dt><a class="_sp_each_title" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;sid1=101&amp;oid=020&amp;aid=0000208705" onclick="return goOtherCR(this, 'a=nws*e.tit&amp;r=12&amp;i=880000A9_000000000000000000208705&amp;g=&amp;u='+urlencode(this.href));" target="_blank" title="[오늘의 생활뉴스]分家해도 그 해에는 소득공제">[오늘의 생활뉴스]分家해도 그 해에는 <strong class="hl">소득</strong>공제</a></dt> <dd class="txt_inline"><span class="_sp_each_source">동아일보</span> <span class="bar"></span> 2003.10.08. <span class="bar"></span> <a class="_sp_each_url" href="https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;sid1=101&amp;oid=020&amp;aid=0000208705" onclick="return goOtherCR(this, 'a=nws*e.nav&amp;r=12&amp;i=880000A9_000000000000000000208705&amp;u='+urlencode(this.href));" target="_blank">네이버뉴스</a> <div class="scial" id="sp_nws12_base"> <span class="bar"></span> <a class="bt_scial2 naver-splugin" data-oninitialize="splugin_oninitialize('sp_nws12');" data-style="unity-v2" h

In [31]:
old_article = requests.get("https://news.naver.com/main/read.nhn?mode=LSD&amp;mid=sec&amp;sid1=101&amp;oid=020&amp;aid=0000208705")

In [37]:
old_article.status_code

404

한겨레 기사

In [52]:
len(all_articles)

4

In [43]:
all_articles

[]

In [47]:
for a in articles:
    print(a.select_one("span._sp_each_source").text)

매일일보
머니S언론사 선정
브릿지경제
오마이뉴스언론사 선정
아시아경제언론사 선정
아시아타임즈
뉴스1언론사 선정
쿠키뉴스
NSP통신
전남일보


In [50]:
"아시아경제언론사"

'아시아경제언론사'

In [54]:
articles[0]

<li id="sp_nws291"> <div class="thumb"><a class="sp_thmb thmb80" href="http://www.m-i.kr/news/articleView.html?idxno=733920" onclick="return goOtherCR(this, 'a=nws*f.img&amp;r=291&amp;i=88155e4f_000000000000000000418611&amp;g=5385.0000418611&amp;u='+urlencode(this.href));" target="_blank"><img alt='국회로 간 이재명 "죽' class="" height="80" onerror="this.parentNode.style.display='none';" src="https://search.pstatic.net/common/?src=https%3A%2F%2Fimgnews.pstatic.net%2Fimage%2Forigin%2F5385%2F2020%2F07%2F30%2F418611.jpg&amp;type=ofullfill80_80_q75_re2" width="80"/><span class="thmb_v"></span></a></div> <dl> <dt><a class="_sp_each_url _sp_each_title" href="http://www.m-i.kr/news/articleView.html?idxno=733920" onclick="return goOtherCR(this, 'a=nws*f.tit&amp;r=291&amp;i=88155e4f_000000000000000000418611&amp;g=5385.0000418611&amp;u='+urlencode(this.href));" target="_blank" title='국회로 간 이재명 "죽음 문턱서 생환" 감사에 환호성'>국회로 간 이재명 "죽음 문턱서 생환" 감사에 환호성</a></dt> <dd class="txt_inline"><span class="_sp_each_source

In [59]:
articles[0].select_one("dd.txt_inline").text

'매일일보  4일 전   보내기  '

In [69]:
len(data['title'])

5000

In [70]:
df = pd.DataFrame(data)

In [86]:
df.head(10)

,title,outlet,date,pick
0,"'저도 임차인입니다' 용혜인 ""집값 낮추는 국회 만들자""",UPI뉴스,7분 전,False
1,"홍천군, 재난기본소득 200억5950만원 지급…지급률 97.4%",MTN,47분 전,False
2,"이장원 한국고용노사관계학회장 ""포스트 코로나19, 고용보험 넘어선 사회보장...",월간노동법률,49분 전,False
3,동료에 상품권 나눠준 직원 징계 놓고 경기도-남양주시 충돌,연합뉴스,59분 전 네이버뉴스,False
4,"21대 국회 농해수위, 반성 없는 농정당국에 질타",한국농정신문,1시간 전,False
5,홍천군 재난기본소득 200억 지급…지급률 97%,이뉴스투데이,2시간 전,False
6,홍천군 재난기본소득 지급률 97.4% 완료,일간투데이,3시간 전,False
7,"홍천군, 재난기본소득 추가지급 마감…지급률 97.4% 달성",쿠키뉴스,3시간 전,False
8,양극화 해소를 위한 새로운 일자리 모델의 기획,노동과희망,4시간 전,False
9,위기에 더 커진 ‘몰빵’…스크린 독점은 ‘무방비’,더리더,4시간 전,False


In [72]:
df.pick.sum()

723

In [75]:
df[df.outlet.apply(lambda x: x in outlets_of_interest)].shape

(57, 4)

In [80]:
df.to_csv("../data/processed/naver_titles_5000.csv", index=False)

In [77]:
np.sum(parse_time_per_page)

783.1217935085297

In [79]:
783.1217935085297 / 60

13.052029891808827

In [81]:
b = requests.get("https://search.naver.com/search.naver?where=news&query=%22%EA%B8%B0%EB%B3%B8%EC%86%8C%EB%93%9D%22&sm=tab_opt&sort=1&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so%3Add%2Cp%3Aall%2Ca%3Aall&mynews=1&refresh_start=0&related=0")

In [82]:
html = BeautifulSoup(b.text, "html.parser")
articles = html.select("ul.type01 > li")

In [85]:
for aa in articles:
    print(aa.select_one("span._sp_each_source").text)

업코리아
UPI뉴스
MTN
월간노동법률
연합뉴스
한국농정신문
이뉴스투데이
일간투데이
쿠키뉴스
노동과희망


# 다음

In [111]:
search_keyword = '"기본소득"'
cp_dict = {
    '조선일보': '16d4PV266g2j-N3GYq',
    '한겨레': '16nzyJHdH5ORpabfqG',
    '경향신문': '16akMkKFDu6n8GTzZr',
    '동아일보': '16bOiOx4gG2S18EPLj',
}

cp_key = cp_dict['한겨레']
page_num=1

daum_url = f"https://search.daum.net/search?w=news&q={search_keyword}&cp={cp_key}&DA=PGD&p={page_num}&cluster=n"

In [112]:
r = requests.get(daum_url)
html = BeautifulSoup(r.text, "html.parser")
articles = html.select("ul#newsResultUL > li")

In [120]:
for i, ar in enumerate(articles):
    article_url = ar.select_one('a.f_nb')['href']
    article_res = requests.get(article_url)
    article_html = BeautifulSoup(article_res.text, "html.parser")
    title = article_html.select_one("h3.tit_view").text
    date = article_html.select_one("span.num_date").text
    body = article_html.select_one("div.news_view").text
    break

In [121]:
title

'이재명·원희룡, 기본소득 두고 "맞닿는 점 있다면 경험 교류"'

In [122]:
date

'2020.07.30. 10:36'

In [124]:
body.strip()

'기본소득 도입 협력에 공감대 \n\n\n\n\n\n            이재명 경기지사(왼쪽)와 원희룡 제주지사가 30일 오전 국회 의원회관에서 열린 ‘국회 기본소득 연구포럼 창립총회 및 세미나’에서 악수하고 있다. 연합뉴스\n           \n\n 여야 유력 대선주자인 이재명 경기지사, 원희룡 제주지사가 30일 국회에서 만나 기본소득 이슈에 대한 입장을 내놨다. 두 사람은 이날 국회 의원회관에서 열린 ‘국회 기본소득 연구포럼’ 창립총회에 참석해 기본소득 도입을 위해 협력해야 한다는 데 공감대를 표했다.\n이 지사는 “기본소득은 단순한 복지정책이 아닌 복지적인 성격을 띠는 경제정책”이라며 “자본주의 시스템이 불평등과 격차, 소비 수요 부족에 따른 저성장 등 한계를 노정하고 있다. 소비 수요를 정부 지원으로 늘리는 데는 기본소득 만한 게 없다”고 말했다. \n원 지사는 “요즘 기본소득이 네 것이다, 내 것이다 하는데 결국 실현 가능한 한국형 모델을 만들어내느냐가 궁극적인 고민이다. 디지털 대전환 가속화 시대에 어떻게 국민의 기회와 역량을 키우며 소득 보장을 결합할 것인지를 고민하고 있다”고 했다. 이어 “이 지사와 내가 하는 부분이 맞닿는 점이 있다면 경험을 교류하며 실현·지속 가능한 방향을 함께 추구하도록 하겠다”고 밝혔다. \n여야 대선주자 후보군인 두 사람은 최근 국회에서 열리는 각종 토론회에 적극 참여하면서 존재감을 내비치고 있다. 대표적 기본소득론자인 이 지사는 지난달 미래통합당에서 기본소득을 논의해야 한다는 주장이 나오자, “단기목표 연 50만원, 중기목표 연 100만원, 장기목표 연 200만∼600만원 등 장단기별 목표를 두고 실시하면 기본소득은 어려울 것이 없다”며 재원 구상방안 등을 제시한 바 있다. 원 지사도 지난달 ‘사회안전망 4.0과 기본소득제’ 정책토론회에 참석해 김종인 통합당 비대위원장과 마주 앉고 “한국식 기본소득을 고민해야 한다”고 밝혔다. \n국회 기본소득연구포럼은 기본소득제 도입을 위한 정책로드맵 구축과 실행방안 연구를 목

In [160]:
paragraphs = article_html.select("div#harmonyContainer.article_view > section > p")
text = '\n\n'.join([p.text.strip() for p in paragraphs])

In [163]:
summary = article_html.select_one("strong.summary_view").text.strip()

In [164]:
summary

' 기본소득 도입 협력에 공감대 '

In [138]:
cnt_text = html.select_one("span#resultCntArea").text
cnt = cnt_text.split("/")[1]

int(''.join(re.findall('\d+', cnt)))

3480

In [139]:
cnt_text

'1-10 / 약 3,480건'

In [146]:
import os

for f in os.listdir("../data"):
    print(f.__dict__)
    break

AttributeError: 'str' object has no attribute '__dict__'